# 데이터 다루기 (2) — 당뇨병 데이터로 기초통계 실습

본 노트북은 슬라이드 내용을 **코드로 따라 하는 실습**입니다.

## 목표
- 숫자형/범주형 구분
- 평균·중앙값·표준편차 계산
- `describe()` 요약과 간단 시각화(히스토그램/박스플롯/막대)

## 0. 준비: 라이브러리 임포트 & 데이터 로드
- 로컬에 `diabetes.csv`가 있으면 해당 파일을 사용하고,
- 없으면 **동일 스키마의 샘플 데이터**를 생성하여 사용합니다.

열 스키마:
`Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome`

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

path = Path('diabetes.csv')
if path.exists():
    df = pd.read_csv(path)
else:
    # 샘플 데이터 생성(재현을 위해 고정 시드)
    rng = np.random.default_rng(42)
    n = 768
    pregnancies = rng.integers(0, 14, size=n)
    glucose = rng.normal(120, 30, size=n).clip(50, 250)
    bp = rng.normal(70, 12, size=n).clip(30, 122)
    skin = rng.normal(25, 8, size=n).clip(0, 99)
    insulin = rng.lognormal(mean=4.0, sigma=0.6, size=n).clip(0, 900)
    bmi = rng.normal(32, 6, size=n).clip(15, 60)
    dpf = rng.gamma(shape=2.0, scale=0.2, size=n).clip(0.05, 3.0)
    age = rng.integers(21, 80, size=n)
    # Outcome 확률을 혈당/나이에 약간 의존하도록 설정
    logits = (glucose - 110)/25 + (age - 45)/20
    probs = 1/(1+np.exp(-logits))
    outcome = (rng.random(n) < probs).astype(int)
    df = pd.DataFrame({
        'Pregnancies': pregnancies,
        'Glucose': glucose.round(0),
        'BloodPressure': bp.round(0),
        'SkinThickness': skin.round(0),
        'Insulin': insulin.round(0),
        'BMI': bmi.round(1),
        'DiabetesPedigreeFunction': dpf.round(3),
        'Age': age,
        'Outcome': outcome,
    })
    df.to_csv(path, index=False)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,120.0,69.0,25.0,47.0,32.4,0.342,33,1
1,10,160.0,70.0,26.0,68.0,29.8,0.498,32,0
2,9,146.0,91.0,21.0,140.0,38.6,0.303,56,0
3,6,145.0,42.0,28.0,74.0,30.0,0.753,58,1
4,6,137.0,46.0,19.0,19.0,29.5,1.003,53,1


## 1. 숫자형 vs 범주형 변수 구분
- **숫자형**: 연산 가능한 열(Glucose, BMI, Age 등)
- **범주형**: 라벨/그룹을 나타내는 열(Outcome)

Outcome을 범주형으로 명시하면 그룹 연산/표시가 편해집니다.

In [ ]:
df['Outcome'] = df['Outcome'].astype('category')
num_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
num_cols, cat_cols

## 2. 평균(mean)·중앙값(median)·표준편차(std)
Glucose를 예로 들어 계산해 봅니다.

In [ ]:
mean_glucose = df['Glucose'].mean()
median_glucose = df['Glucose'].median()
std_glucose = df['Glucose'].std()
skew_glucose = df['Glucose'].skew()
mean_glucose, median_glucose, std_glucose, skew_glucose

해석 가이드
- 평균과 중앙값이 비슷하면 분포가 **대칭**에 가깝습니다.
- `skew()`가 양수면 오른쪽 꼬리가 긴 분포(평균 > 중앙값 경향), 음수면 반대.

## 3. describe()로 요약
숫자형 전체 요약과 범주형 빈도/고유값 표시를 확인합니다.

In [ ]:
desc_num = df.describe()
desc_cat = df.describe(include='category')
desc_num, desc_cat

## 4. 범주형 요약 — Outcome 빈도와 비율

In [ ]:
counts = df['Outcome'].value_counts()
ratios = df['Outcome'].value_counts(normalize=True).round(3)
counts, ratios

## 5. 시각화 — 히스토그램(Glucose)
- 주의: 시각화는 **matplotlib**만 사용하며, 한 셀에 한 차트만, 색상 지정 없이 그립니다.

In [ ]:
plt.figure()
plt.hist(df['Glucose'].dropna(), bins=30, density=False)
plt.title('Glucose 분포 (히스토그램)')
plt.xlabel('Glucose')
plt.ylabel('빈도')
plt.show()

## 6. 시각화 — 박스플롯(BMI)

In [ ]:
plt.figure()
plt.boxplot(df['BMI'].dropna(), vert=False)
plt.title('BMI 분포 (박스플롯)')
plt.xlabel('BMI')
plt.show()

## 7. 시각화 — Outcome 빈도(막대)
0: 비당뇨, 1: 당뇨 (샘플 생성 데이터에서는 통계적 분포가 실제와 다를 수 있습니다.)

In [ ]:
plt.figure()
x = counts.index.astype(str)
y = counts.values
plt.bar(x, y)
plt.title('Outcome 빈도')
plt.xlabel('Outcome')
plt.ylabel('건수')
plt.show()

## 8. 그룹 비교 — Outcome별 요약
Outcome(0/1) 그룹에 따라 주요 수치형 변수의 평균/중앙값/표준편차를 비교합니다.

In [ ]:
group_stats = df.groupby('Outcome')[['Glucose','BMI','Age']].agg(['mean','median','std'])
group_stats

## 9. 평균 vs 중앙값 관계 확인 예시(Glucose)
분포가 오른쪽 꼬리가 길면 평균이 중앙값보다 커지는 경향을 수치로 확인합니다.

In [ ]:
comp = pd.DataFrame({
    'mean_glucose': [df['Glucose'].mean()],
    'median_glucose': [df['Glucose'].median()],
    'std_glucose': [df['Glucose'].std()],
    'skew_glucose': [df['Glucose'].skew()],
})
comp

## 10. 선택 과제(옵션)
- `describe(include='all')` 결과를 이용해 **범주형**(Outcome)의 최빈값을 찾아보세요.
- `Age` 히스토그램을 그리고 분포의 `skew()`를 계산해 해석해 보세요.
- `Outcome`별 `Glucose` 박스플롯을 두 개의 셀로 나눠 각각 그려보세요(한 셀=한 차트).
- 코호트가 없으므로, 임의로 `Age`를 구간화(예: 20대/30대/40대…)하여 `Outcome` 비율을 비교해 보세요.